In [1]:
%reload_ext autoreload
%autoreload 2

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt

import keras
from keras.layers import LeakyReLU, UpSampling1D, Input, Reshape, Activation, Lambda, AveragePooling1D
from keras.layers import Convolution2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.models import load_model
import random

# From pypng
import png
#from sklearn.p

FONTSIZE = 18
FIGURE_SIZE = (10,4)
FIGURE_SIZE2 = (10,10)

# Configure parameters
plt.rcParams.update({'font.size': FONTSIZE})

####################################
# 
import re

##################
# Default tick label size
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24



Using TensorFlow backend.


In [2]:
#Function for generating list of images of respective parts.
def make_df(file, coloumn_name, code, new_coloumn_entry):
    image_names = []
    check_for = file[coloumn_name] == code
    file_data = file[check_for]
    file_data['Lesion_typename'] = new_coloumn_entry
    return(file_data)

#Function for generating training, validation and test sets list of images of respective parts.
def sets(image_names):
    train_length = len(image_names) * 0.7
    int_train_length = int(train_length)
    val_length = int_train_length + len(image_names) * 0.15
    int_val_length = int(val_length)
    trainset = image_names[:int_train_length]
    valset = image_names[int_train_length:int_val_length]
    testset = image_names[int_val_length:]
    return (trainset, valset, testset)

#Function for generating lists of imagenames and labels for trainset, valset and testset.
def make_list(Full_list, data_length):
    image_name = []
    image_label = []
    #Storing annotated image file names in a list (image_name).
    for i in range (data_length):
        image_name.append(Full_list['File_name'].values[i])
    #Storing annotated image label names in a list (image_label).
    for i in range (data_length):
        image_label.append(Full_list['Coarse_lesion_type'].values[i])
    return (image_name, image_label)

#Function for reading each image file and returning a 3-dimensional array.
def Read_image(image_name):
    read_image = png.Reader(image_name)
    image_copy = read_image.read()
    image_in_2d = np.vstack(map(np.uint8, image_copy[2]))
    image_in_3d = np.reshape(image_in_2d, (image_copy[0],image_copy[1],image_copy[3]['planes'])) / 512.0
    return image_in_3d

#Function for reading each image file from given directory using given image name list and returning a array.
def Image_directory(directory, image_name):
    print(directory, image_name)
    #Get all of the file names
    all_images = sorted(os.listdir(directory))
    list_of_images = [Read_image(directory + "/" + i) for i in all_images if re.search(image_name, i) ]    
    return np.array(list_of_images, dtype=np.float32)



In [3]:
#Function which takes measurement coordinates from csv file and compute mid-point of bounding box and height and width
def target_point(Measurement_coordinates):
    split_Mesur_list = []
    x_pt = []
    y_pt = []
    
    split_Mesur_list = Measurement_coordinates.split(', ')
    x_ind = [0, 2, 4, 6]
    y_ind = [1, 3, 5, 7]
    
    #List for respective coordinate points.
    for i in x_ind:
        x_pt.append(float(split_Mesur_list[i]))
    for j in y_ind:
        y_pt.append(float(split_Mesur_list[j]))
    
    #Height and width of bounding box
    height = max(y_pt) - min(y_pt)
    width = max(x_pt) - min(x_pt)
    
    #Mid point estimation.
    x_mid_1 = (x_pt[0] + x_pt[1])/2
    y_mid_1 = (y_pt[0] + y_pt[1])/2
    x_mid_2 = (x_pt[2] + x_pt[3])/2
    y_mid_2 = (y_pt[2] + y_pt[3])/2
    x_mid = (x_mid_1 + x_mid_2)/2
    y_mid = (y_mid_1 + y_mid_2)/2
    
    return x_mid, y_mid, height, width

#Function which takes measurement coordinates and grid size in number of pixels and returns target label of respective shape.
def gen_target(Measurement_coordinates, grid_size = 16):
    #Computing mid point, height and width of bounding box.
    x_mid, y_mid, height, width = target_point(Measurement_coordinates)
    #Total number of grids, using given grid size.
    grids = int(512/grid_size)
    
    #Locating grid which has midpoint.
    x_grid = int(x_mid/grid_size)
    y_grid = int(y_mid/grid_size)

    #mid points with respect to each grid
    x_mid_grid = (x_mid/grid_size) - x_grid
    y_mid_grid = (y_mid/grid_size) - y_grid    

    
    #initializing target label as all zeros.
    target = np.zeros((grids,grids,5))
    #Defining and filling up list with "Pc", "bx", "by", "bh", and "bw"
    coord_list = []
    coord_list.append(1)
    coord_list.append(x_mid_grid)
    coord_list.append(y_mid_grid)
    coord_list.append(height)
    coord_list.append(width)
    
    #mid point located grid is filled with the above information.
    for i in range(len(coord_list)):
        target[x_grid][y_grid][i] = coord_list[i]
    
    return target

#Function for generating target labels for trainset, valset and testset.
def make_yolo(inlist, grid_size):
    image_name = []
    yolo_label = []
    data_length = inlist.shape[0]
    #Storing annotated image file names in a list (image_name).
    for i in range (data_length):
        image_name.append(inlist['File_name'].values[i])
    #Storing annotated image label names in a list (image_label).
    for i in range (data_length):
        Measurement_coordinates = inlist['Measurement_coordinates'].values[i]
        yolo_label.append(gen_target(Measurement_coordinates, grid_size))
    return (image_name, yolo_label)


In [9]:
me, you = make_yolo(trainset_Bone, 16)

#trainset_Bone.shape[0]

In [4]:
#Reading file with file name and labels.

#Storing the images in directory.
directory = '/home2/potd0000/project/Key_slices/'

#data information extraction from csv file
Entire_image_list = pd.read_csv(r'/home2/potd0000/project/Dl_info.csv')
#Removing all other coloumns and only selecting Filename, Coarse_lesion_type and, Image_size to be there in dataframe.
list_filename_n_type = Entire_image_list[['File_name', 'Coarse_lesion_type', 'Image_size', 'Measurement_coordinates', 'Bounding_boxes', 'Lesion_diameters_Pixel_', 'Normalized_lesion_location']]

#Eliminating 40 image names which is not of size '512 x 512'
check_for_size = list_filename_n_type['Image_size'] == '512, 512'
list_filename_n_type = list_filename_n_type[check_for_size]


Bone_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 1, 'Bone')
Abdomen_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 2, 'Abdomen')
Mediastinum_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 3, 'Mediastinum')
Liver_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 4, 'Liver')
Lung_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 5, 'Lung')
Kidney_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 6, 'Kidney')
Soft_tissue_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 7, 'Soft_tissue')
Pelvis_image_names = make_df(list_filename_n_type, 'Coarse_lesion_type', 8, 'Pelvis')


trainset_Bone, valset_Bone, testset_Bone = sets(Bone_image_names)
trainset_Abdomen, valset_Abdomen, testset_Abdomen = sets(Abdomen_image_names)
trainset_Mediastinum, valset_Mediastinum, testset_Mediastinum = sets(Mediastinum_image_names)
trainset_Liver, valset_Liver, testset_Liver = sets(Liver_image_names)
trainset_Lung, valset_Lung, testset_Lung = sets(Lung_image_names)
trainset_Kidney, valset_Kidney, testset_Kidney = sets(Kidney_image_names)
trainset_Soft_tissue, valset_Soft_tissue, testset_Soft_tissue = sets(Soft_tissue_image_names)
trainset_Pelvis, valset_Pelvis, testset_Pelvis = sets(Pelvis_image_names)

training_name_df = pd.DataFrame(trainset_Bone)
training_name_df = training_name_df.append(trainset_Abdomen)
training_name_df = training_name_df.append(trainset_Mediastinum)
training_name_df = training_name_df.append(trainset_Liver)
training_name_df = training_name_df.append(trainset_Lung)
training_name_df = training_name_df.append(trainset_Kidney)
training_name_df = training_name_df.append(trainset_Soft_tissue)
training_name_df = training_name_df.append(trainset_Pelvis)

validation_name_df = pd.DataFrame(valset_Bone)
validation_name_df = validation_name_df.append(valset_Abdomen)
validation_name_df = validation_name_df.append(valset_Mediastinum)
validation_name_df = validation_name_df.append(valset_Liver)
validation_name_df = validation_name_df.append(valset_Lung)
validation_name_df = validation_name_df.append(valset_Kidney)
validation_name_df = validation_name_df.append(valset_Soft_tissue)
validation_name_df = validation_name_df.append(valset_Pelvis)

testing_name_df = pd.DataFrame(testset_Bone)
testing_name_df = testing_name_df.append(testset_Abdomen)
testing_name_df = testing_name_df.append(testset_Mediastinum)
testing_name_df = testing_name_df.append(testset_Liver)
testing_name_df = testing_name_df.append(testset_Lung)
testing_name_df = testing_name_df.append(testset_Kidney)
testing_name_df = testing_name_df.append(testset_Soft_tissue)
testing_name_df = testing_name_df.append(testset_Pelvis)

trainset_length = len(training_name_df)
valset_length = len(validation_name_df)
testset_length = len(testing_name_df)


training_image_name, training_image_label = make_list(training_name_df, trainset_length)
validation_image_name, validation_image_label = make_list(validation_name_df, valset_length)
testing_image_name, testing_image_label = make_list(testing_name_df, testset_length)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
yolo_train_bone_ins, yolo_train_bone_outs = make_yolo(trainset_Bone, 16)
yolo_val_bone_ins, yolo_val_bone_outs = make_yolo(valset_Bone, 16)
yolo_test_bone_ins, yolo_test_bone_outs = make_yolo(testset_Bone, 16)


In [12]:

train_image_data_list = []

for names in yolo_train_bone_ins:
    train_image_data_list.extend(Image_directory(directory, names))
input_train_image = np.array(train_image_data_list)
input_train_image.shape


/home2/potd0000/project/Key_slices/ 000010_02_02_100.png
/home2/potd0000/project/Key_slices/ 000020_02_01_062.png
/home2/potd0000/project/Key_slices/ 000122_04_02_150.png
/home2/potd0000/project/Key_slices/ 000193_01_02_469.png
/home2/potd0000/project/Key_slices/ 000193_02_02_477.png
/home2/potd0000/project/Key_slices/ 000193_03_01_458.png
/home2/potd0000/project/Key_slices/ 000207_05_02_444.png
/home2/potd0000/project/Key_slices/ 000207_06_01_089.png
/home2/potd0000/project/Key_slices/ 000207_06_02_444.png
/home2/potd0000/project/Key_slices/ 000207_06_02_443.png
/home2/potd0000/project/Key_slices/ 000207_07_01_223.png
/home2/potd0000/project/Key_slices/ 000207_07_01_444.png
/home2/potd0000/project/Key_slices/ 000250_02_01_094.png
/home2/potd0000/project/Key_slices/ 000250_03_01_093.png
/home2/potd0000/project/Key_slices/ 000280_03_02_501.png
/home2/potd0000/project/Key_slices/ 000281_03_01_291.png
/home2/potd0000/project/Key_slices/ 000281_04_01_022.png
/home2/potd0000/project/Key_sli

(172, 512, 512, 3)

In [31]:
np.save('yolo_bone_input_train_image_data', input_train_image)

In [6]:
yolo_bone_input_train_image_saved = np.load('yolo_bone_input_train_image_data.npy')

In [32]:

val_image_data_list = []

for names in yolo_val_bone_ins:
    val_image_data_list.extend(Image_directory(directory, names))
input_val_image = np.array(val_image_data_list)
input_val_image.shape

/home2/potd0000/project/Key_slices/ 003209_03_03_320.png
/home2/potd0000/project/Key_slices/ 003209_05_02_261.png
/home2/potd0000/project/Key_slices/ 003209_05_02_329.png
/home2/potd0000/project/Key_slices/ 003209_06_01_267.png
/home2/potd0000/project/Key_slices/ 003209_06_01_334.png
/home2/potd0000/project/Key_slices/ 003210_01_01_129.png
/home2/potd0000/project/Key_slices/ 003226_01_02_528.png
/home2/potd0000/project/Key_slices/ 003285_02_02_451.png
/home2/potd0000/project/Key_slices/ 003327_01_02_177.png
/home2/potd0000/project/Key_slices/ 003347_02_01_020.png
/home2/potd0000/project/Key_slices/ 003347_02_01_084.png
/home2/potd0000/project/Key_slices/ 003347_02_01_123.png
/home2/potd0000/project/Key_slices/ 003406_01_01_103.png
/home2/potd0000/project/Key_slices/ 003406_01_01_094.png
/home2/potd0000/project/Key_slices/ 003420_01_01_395.png
/home2/potd0000/project/Key_slices/ 003420_02_01_412.png
/home2/potd0000/project/Key_slices/ 003420_02_01_372.png
/home2/potd0000/project/Key_sli

(36, 512, 512, 3)

In [33]:
np.save('yolo_bone_input_val_image_data', input_val_image)

In [7]:
yolo_bone_input_val_image_saved = np.load('yolo_bone_input_val_image_data.npy')

In [34]:

test_image_data_list = []

for names in yolo_test_bone_ins:
    test_image_data_list.extend(Image_directory(directory, names))
input_test_image = np.array(test_image_data_list)
input_test_image.shape

/home2/potd0000/project/Key_slices/ 003663_02_03_117.png
/home2/potd0000/project/Key_slices/ 003722_02_02_227.png
/home2/potd0000/project/Key_slices/ 003722_02_03_477.png
/home2/potd0000/project/Key_slices/ 003722_02_03_417.png
/home2/potd0000/project/Key_slices/ 003722_03_01_225.png
/home2/potd0000/project/Key_slices/ 003722_03_01_417.png
/home2/potd0000/project/Key_slices/ 003722_03_01_480.png
/home2/potd0000/project/Key_slices/ 003723_02_03_509.png
/home2/potd0000/project/Key_slices/ 003723_03_01_056.png
/home2/potd0000/project/Key_slices/ 003723_03_01_100.png
/home2/potd0000/project/Key_slices/ 003730_01_03_224.png
/home2/potd0000/project/Key_slices/ 003730_01_03_229.png
/home2/potd0000/project/Key_slices/ 003877_01_02_473.png
/home2/potd0000/project/Key_slices/ 003882_02_02_155.png
/home2/potd0000/project/Key_slices/ 003980_01_02_254.png
/home2/potd0000/project/Key_slices/ 004000_01_01_203.png
/home2/potd0000/project/Key_slices/ 004000_02_01_175.png
/home2/potd0000/project/Key_sli

(38, 512, 512, 3)

In [35]:
np.save('yolo_bone_input_test_image_data', input_test_image)

In [8]:
yolo_bone_input_test_image_saved = np.load('yolo_bone_input_test_image_data.npy')

In [9]:
ins = yolo_bone_input_train_image_saved
outs = np.array(yolo_train_bone_outs)
ins_validation = yolo_bone_input_val_image_saved
outs_validation = np.array(yolo_val_bone_outs)
ins_test = yolo_bone_input_test_image_saved
outs_test = np.array(yolo_test_bone_outs)

In [62]:
#Constructing Network

#I am using three convolution layers with 100 filters.
#I am using three convolution layers with 200 filters.
#I am using three convolution layers with 300 filters.
#I am using three convolution layers with 500 filters.
#I am using regularization with value 0
#To bring down the number of parameters, I am using 1 x 1 strides in two Maxpooling layers.

def construct_model(input_shape, lambda_regularization = 0):
    model = Sequential()
    
    model.add(Convolution2D(filters = 100,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 2,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C0'))

    model.add(MaxPooling2D(pool_size = (3, 3), strides = (1, 1), name = 'C1'))
    
    model.add(Convolution2D(filters = 100,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C2'))
    
    model.add(Convolution2D(filters = 100,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C3'))
    
    model.add(Convolution2D(filters = 200,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 2,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C4'))

    model.add(MaxPooling2D(pool_size = (3, 3), strides = (1, 1), name = 'C5'))
    
    model.add(Convolution2D(filters = 200,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C6'))

    model.add(Convolution2D(filters = 200,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C7'))

    model.add(Convolution2D(filters = 300,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 2,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C8'))

    model.add(Convolution2D(filters = 300,
                           input_shape = input_shape,
                           kernel_size = (5, 5),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C9'))

    model.add(Convolution2D(filters = 300,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C10'))
    
    model.add(Convolution2D(filters = 300,
                           input_shape = input_shape,
                           kernel_size = (3, 3),
                           strides = 1,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'elu',
                           name = 'C11'))
        
    model.add(Convolution2D(filters = 5,
                           input_shape = input_shape,
                           kernel_size = (1, 1),
                           strides = 2,
                           padding = 'same',
                           use_bias = True,
                           kernel_initializer = 'random_uniform',
                           bias_initializer = 'random_uniform',
                           kernel_regularizer = keras.regularizers.l2(lambda_regularization),
                           bias_regularizer = keras.regularizers.l2(lambda_regularization),
                           activation = 'softmax',
                           name = 'output'))
        
    
    # The optimizer determines how the gradient descent is to be done
    opt = keras.optimizers.Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, 
                            epsilon = None, decay = 0.0, amsgrad = False)
    
    model.compile(loss = 'binary_crossentropy', optimizer = opt, 
                  metrics = ['accuracy'])
    print(model.summary())
    return model

In [63]:
model = construct_model(ins.shape[1:])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C0 (Conv2D)                  (None, 256, 256, 100)     2800      
_________________________________________________________________
C1 (MaxPooling2D)            (None, 254, 254, 100)     0         
_________________________________________________________________
C2 (Conv2D)                  (None, 254, 254, 100)     90100     
_________________________________________________________________
C3 (Conv2D)                  (None, 254, 254, 100)     90100     
_________________________________________________________________
C4 (Conv2D)                  (None, 127, 127, 200)     180200    
_________________________________________________________________
C5 (MaxPooling2D)            (None, 125, 125, 200)     0         
_________________________________________________________________
C6 (Conv2D)                  (None, 125, 125, 200)     360200    
__________

In [64]:
def training_set_generator(ins, outs, batch_size = 10,
                      input_name = 'input_input', 
                      output_name = 'output'):
    while True:
        example_indices = random.choices(range(ins.shape[0]), k=batch_size)
        yield({input_name: ins[example_indices,:,:,:]},
             {output_name: outs[example_indices]})


In [ ]:
generator = training_set_generator(ins,outs, batch_size = 10, 
                                  input_name = 'C0_input')

#Stopping criteria for epoch termination with patience = 10
early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                               min_delta = 0, patience = 10, 
                                               verbose = 0, mode = 'auto', 
                                               baseline = None, 
                                               restore_best_weights = True)

#Total training set has 172 images and batch size is 10.
#Hence 18 steps per epoch will cover entire training dataset.
model.fit_generator(generator, epochs = 10, steps_per_epoch = 18, 
                    validation_data=(ins_validation, outs_validation))

Epoch 1/10
 4/18 [=====>........................] - ETA: 28:20 - loss: 0.2519 - acc: 0.9119

In [18]:
model.save('yolo_model_bhairav-1.h5')

In [19]:
model = keras.models.load_model('yolo_model_bhairav-1.h5')

In [19]:
pred = model.predict(ins_test)

In [21]:
pred[0]

array([[[4.17996570e-02, 3.58160995e-02, 3.45983021e-02, 1.47575989e-01,
         1.23508200e-01],
        [5.53203933e-03, 3.23418435e-03, 4.56377584e-03, 3.42267789e-02,
         3.11615486e-02],
        [3.96712078e-03, 2.30848766e-03, 3.17956414e-03, 2.51502469e-02,
         2.68612951e-02],
        ...,
        [3.96324322e-03, 2.30699033e-03, 3.18264705e-03, 2.51875278e-02,
         2.68876292e-02],
        [5.41210035e-03, 3.54228611e-03, 4.40082140e-03, 2.79009622e-02,
         3.22699733e-02],
        [5.20220175e-02, 3.44998278e-02, 4.94083129e-02, 8.97093862e-02,
         1.00782409e-01]],

       [[5.37871290e-03, 4.56834212e-03, 3.79505754e-03, 3.41265686e-02,
         2.83698626e-02],
        [1.13858907e-04, 6.76684140e-05, 7.81247363e-05, 2.60124728e-03,
         2.43295357e-03],
        [6.30896466e-05, 3.75346171e-05, 3.92024813e-05, 1.59033609e-03,
         1.82374823e-03],
        ...,
        [6.19839630e-05, 3.72336908e-05, 3.92834263e-05, 1.65177754e-03,
        

In [23]:
ins_test[0]

array([[[0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        ...,
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062]],

       [[0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        ...,
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062]],

       [[0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        ...,
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062]],

       ...,

       [[0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0.11914062, 0.11914062],
        [0.11914062, 0